# Prepare Titanic Model for API

Sources:
1. https://towardsdatascience.com/create-an-api-to-deploy-machine-learning-models-using-flask-and-heroku-67a011800c50?gi=30b632ffd17d
2. https://blog.cambridgespark.com/deploying-a-machine-learning-model-to-the-web-725688b851c7




## Import data, create and train model

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression


In [2]:
# create df
train = pd.read_csv('titanic_train.csv')  # change file path
# drop null values
train.dropna(inplace=True)

# features and target
target = 'Survived'
features = ['Pclass', 'Age', 'SibSp', 'Fare']# X matrix, y vector
X = train[features]
y = train[target]# model 
model = LogisticRegression(solver='lbfgs')
model.fit(X, y)
model.score(X, y)

0.7103825136612022

## Save model as pickle

In [3]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

## Run app.py

type "python app.py" in console while in folder

## Test app web interface
Type "flask run" in terminal, once in this directory

## Test app API


In [4]:
# local url
url = 'http://127.0.0.1:5000' 

In [5]:
# sample data
data = {'Pclass': 3
      , 'Age': 2
      , 'SibSp': 1
      , 'Fare': 50}
data = json.dumps(data)

In [6]:
import requests, json

#send_request = requests.post(url, data)
#print(send_request)

In [29]:
# check actual result
#print(send_request.json())

to stop app, press ctrl+c in console

## Then create Procfile for Heroku app

## Then create requirements.txt

Use:
Flask==1.1.1
gunicorn==19.9.0
pandas==0.25.0
requests==2.22.0
scikit-learn==0.21.2
scipy==1.3.1

More generally, can do:
pip freeze > requirements.txt

## Deploy on Heroku

## Check Heroku

In [8]:
# heroku url
heroku_url = 'https://titanic-model-app.herokuapp.com/' # change to your app name# sample data
data = {  'Pclass': 3
             , 'Age': 2
             , 'SibSp': 1
             , 'Fare': 50}
data = json.dumps(data)

In [9]:
# may need to disable firewall for this
send_request = requests.post(heroku_url, data)
print(send_request)

<Response [200]>


In [10]:
print(send_request.json())

{'result': {'results': 1}}
